**Домашнее задание Полны Кудрявцевой, БКЛ182**

In [ ]:
import os
import pandas as pd
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import numpy as np
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer

m = MorphAnalyzer()
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
! pip install pymorphy2

In [ ]:
! pip install python-rake

In [ ]:
! pip install summa

**Мини-корпус**

Для выполнения задания были отобраны четыре статьи с habr.com, по поиску по слову "кино" - на сопутствующие темы. На habr.com, как мне кажется, тэги являются в действительности ключевыми словами, а не просто тэгами по теме - они выделяют важные слова из текста.

Тексты были собраны в .txt файлы, первая строка в которых - ключевые слова с сайта habr.com, вторая - мои ключевые слова, третья - текст.

*Тексты составили примерно 6 тысяч токенов.*

In [ ]:
def get_text_keywords(filename):
    with open(filename, 'r', encoding='maccyrillic') as f:
        filetext = f.read()
    parts = filetext.split('\n')
    orig_keywords = parts[0]
    my_keywords = parts[1]
    text = parts[2]
    return orig_keywords, my_keywords, text

In [ ]:
links = ['https://habr.com/ru/company/timeweb/blog/586582/', 'https://habr.com/ru/company/plarium/blog/389469/',
         'https://habr.com/ru/post/504056/', 'https://habr.com/ru/post/393729/']
list_orig_keywords = []
list_my_keywords = []
texts = []
for filename in os.listdir('/content/drive/MyDrive/Colab Notebooks/Автобрея'):
    if filename.endswith('.txt'):
        print('Скачано: ', filename)
        orig_keywords, my_keywords, text = get_text_keywords('/content/drive/MyDrive/Colab Notebooks/Автобрея/'+filename)
        list_orig_keywords.append(orig_keywords)
        list_my_keywords.append(my_keywords)
        texts.append(text)

Скачано:  Голос VHS: Василий Горчаков об озвучке кино и компьютерных игр.txt
Скачано:  Кино и космос.txt
Скачано:  Экстрасенсорные шпионы из Китая пытаются украсть восторг вашего разума: дроны в кино.txt
Скачано:  Эффект звона в ушах в кино.txt


Тексты и ключевые слова собираю в датафрейм.

In [ ]:
df = pd.DataFrame(links, columns = ['link'])
df['link'] = links
df['original_keywords'] = list_orig_keywords
df['my_keywords'] = list_my_keywords
df['text'] = texts
df

,link,original_keywords,my_keywords,text
0,https://habr.com/ru/company/timeweb/blog/586582/,"чужой, хищник, перевод, VHS, локализация, пере...","горчаков, чужой против хищник, игра конфликт, ...",Недавно один из легендарных переводчиков эпохи...
1,https://habr.com/ru/company/plarium/blog/389469/,"история космонавтика, история кино, кино в кос...","шипенко, пересильд, шкаплеров, новицкий, кино ...",Недавно благополучно завершился полет киноэкип...
2,https://habr.com/ru/post/504056/,"кинодрон, БПЛА, кино, dpi, hollywood","кинодрон, робототехника в кино, motion capture...",История о нашем первом проекте с кинодронами. ...
3,https://habr.com/ru/post/393729/,"тиннитус, работа со звук, кинозвук","звуковой эффект, киноэффект, тиннитус",В фильме с повествованием от первого лица прои...


Можно посмотреть на них поближе.

В моих ключевых словах больше слов из текста, чем в выделенных на сайте - я учитываю имена личностей, которые имеют важную роль в тексте, и беру их фамилии; в ключевых словах на сайте по какой-то непонятной схеме были взяты термины из текста вроде БПЛА, которые упоминались там всего раз или два - такие случаи я не учитывала.

In [ ]:
for i in range(4):
    print('Данные ключ. слова:', df['original_keywords'].iloc[i])
    print('Мои ключ. слова:', df['my_keywords'].iloc[i])
    print('\n')

Данные ключ. слова: чужой, хищник, перевод, VHS, локализация, переводчик, горчаков, интервью, кино
Мои ключ. слова: горчаков, чужой против хищник, игра конфликт, перевод, перевод кино, перевод игра, голос озвучка, озвучивание фильмов


Данные ключ. слова: история космонавтика, история кино, кино в космос, космос в кино, невесомость в кино, невесомость, спецэффект
Мои ключ. слова: шипенко, пересильд, шкаплеров, новицкий, кино в космосе, история кино, спецэффект, научная фантастика, невесомость, вызов, история космонавтика


Данные ключ. слова: кинодрон, БПЛА, кино, dpi, hollywood
Мои ключ. слова: кинодрон, робототехника в кино, motion capture, фотограмметрия, камера


Данные ключ. слова: тиннитус, работа со звук, кинозвук
Мои ключ. слова: звуковой эффект, киноэффект, тиннитус




Я думаю, и мои, и ключевые слова на сайте отобраны вполне нормально.
Термины, которые встречаются в тексте пару раз, я думаю, в конечном итоге тоже важны - и правда могут являться ключевыми словами.
Можно совместить списки и убрать дубли - получается список эталонных ключевых слов в колонке ideal.

In [ ]:
list_ideal = []
for y in range(4):
    small_list = []
    for i in df['my_keywords'].iloc[y].split(', '):
        small_list.append(i)
    for i in df['original_keywords'].iloc[y].split(', '):
        small_list.append(i)
    new_small_list = ', '.join(list(set(small_list)))
    list_ideal.append(new_small_list)
df['ideal'] = list_ideal

In [ ]:
df

,link,original_keywords,my_keywords,text,ideal
0,https://habr.com/ru/company/timeweb/blog/586582/,"чужой, хищник, перевод, VHS, локализация, пере...","горчаков, чужой против хищник, игра конфликт, ...",Недавно один из легендарных переводчиков эпохи...,"игра конфликт, озвучивание фильмов, VHS, перев..."
1,https://habr.com/ru/company/plarium/blog/389469/,"история космонавтика, история кино, кино в кос...","шипенко, пересильд, шкаплеров, новицкий, кино ...",Недавно благополучно завершился полет киноэкип...,"невесомость в кино, вызов, новицкий, спецэффек..."
2,https://habr.com/ru/post/504056/,"кинодрон, БПЛА, кино, dpi, hollywood","кинодрон, робототехника в кино, motion capture...",История о нашем первом проекте с кинодронами. ...,"БПЛА, motion capture, кино, кинодрон, фотограм..."
3,https://habr.com/ru/post/393729/,"тиннитус, работа со звук, кинозвук","звуковой эффект, киноэффект, тиннитус",В фильме с повествованием от первого лица прои...,"работа со звук, звуковой эффект, тиннитус, кин..."


Попробую три метода извлечения ключевых слов - RAKE, TextRank и TF-IDF.

In [ ]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [ ]:
lemma_texts = [normalize_text(text) for text in df['text'].values]

In [ ]:
stop_words.append('который')
stop_words.append('это')
stop_words.append('нужный')

In [ ]:
rake = RAKE.Rake(stop_words)
rake_words_numbers = [rake.run(lemma_text, maxWords=2, minFrequency=2) for lemma_text in lemma_texts]
rake_words = []
for i in range(4):
    rake_words.append(', '.join(word[0] for word in rake_words_numbers[i]))
df['RAKE'] = rake_words

In [ ]:
for i in range(4):
    print('Rake:', df['RAKE'].iloc[i])
    print('Мои ключ. слова:', df['my_keywords'].iloc[i])
    print('\n')

Rake: мочь переводить, честно говорить, переводить фильм, говорить актёр, голос, переводить, актёр, кино, фильм, что-, говорить, человек, какой-, знать, видеть, интересно, перевод, посмотреть, недавно, сеттинг, считать, хотя, участвовать, скажем, думать, наверное, экран, например, ранний, кроме, смотреть, сказать, ощущение, хотеть, картина, слышать, играть, работать, встречаться, помнить, озвучивать, наоборот, стараться, «, »
Мои ключ. слова: горчаков, чужой против хищник, игра конфликт, перевод, перевод кино, перевод игра, голос озвучка, озвучивание фильмов


Rake: настоящий невесомость, скорее весь, снятой космонавт, история кино, космос кадр, актёр, фильм, кадр, ракета, космонавтика, снятой, время, выйти, камера, орбита ”, известный, стать, полёт, стоять, запустить, всё, космос, снять, кино, “ женщина, оказаться, правда, ещё, земля, какой-, трос, видео, второй, иллюминатор, очень, результат, человечество, фон, например, встретить, хотеть, отправиться, мкс, полететь, увы, ”
Мои ключ.

По RAKE, выделяется много глаголов и глагольных групп. Ключевых слов, возможно, слишком много. Многие ключевые слова повторяют друг друга, но, в целом, они соответствуют текстам. 

In [ ]:
textrank_words_numbers = [keywords.keywords(lemma_text, language='russian', 
                  additional_stopwords=stop_words, scores=True, ratio=0.05) for lemma_text in lemma_texts]
textrank_words = []
for i in range(4):
    textrank_words.append(', '.join(word[0] for word in textrank_words_numbers[i]))
df['TextRank'] = textrank_words

In [ ]:
for i in range(4):
    print('TextRank:', df['TextRank'].iloc[i])
    print('Мои ключ. слова:', df['my_keywords'].iloc[i])
    print('\n')

TextRank: фильм, перевод, переводить, игра, голос, актёр, переводчик, новый, свой, кино, год, поэтому, очень, говорить, первый, огромный, появиться, время, особенный, особенно, хороший
Мои ключ. слова: горчаков, чужой против хищник, игра конфликт, перевод, перевод кино, перевод игра, голос озвучка, озвучивание фильмов


TextRank: фильм, космонавт, снятой, снять, орбита стать первый, автоматический космический, камера, год, станция, лунный ракета, полёт, кадр, космос, больший, большой, картина, актёр, время, астронавт, показать, корабль, снимать, весь, космонавтика, выйти, появиться, луна, невесомость, затем, всё, сделать, женщина, технология, сценарий, автоматически, летать, американский
Мои ключ. слова: шипенко, пересильд, шкаплеров, новицкий, кино в космосе, история кино, спецэффект, научная фантастика, невесомость, вызов, история космонавтика


TextRank: дрон, dji, технология, использовать, использоваться, камера, время, съёмка, робот, всё, система, ещё, компания, кино, наш первый п

По TextRank, выделяются, в основном, существительные и именные гуппы. Ключевые слова тоже иногда повторяют друг друга - один смысл разных частей речи. Но результат, на вид, лучше, чем по RAKE.

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(lemma_texts)

tfidf_words = []
for i in range(len(lemma_texts)):
    features = np.array(vectorizer.get_feature_names())
    sorted_indices = np.argsort(tfidf[i].toarray()).ravel()[::-1]
    keywords = features[sorted_indices][:10]
    tfidf_words.append(keywords)

df['TF-IDF'] = tfidf_words

In [ ]:
for i in range(4):
    print('TF-IDF:', ','.join(tfidf_words[i]))
    print('Мои ключ. слова:', df['my_keywords'].iloc[i])
    print('\n')

TF-IDF: фильм,переводить,голос,игра,говорить,актёр,переводчик,перевод,хищник,год
Мои ключ. слова: горчаков, чужой против хищник, игра конфликт, перевод, перевод кино, перевод игра, голос озвучка, озвучивание фильмов


TF-IDF: космос,космонавт,космический,ракета,год,снятой,фильм,камера,орбита,станция
Мои ключ. слова: шипенко, пересильд, шкаплеров, новицкий, кино в космосе, история кино, спецэффект, научная фантастика, невесомость, вызов, история космонавтика


TF-IDF: дрон,робот,dji,камера,кино,использоваться,технология,движение,компания,съёмка
Мои ключ. слова: кинодрон, робототехника в кино, motion capture, фотограмметрия, камера


TF-IDF: звон,ухо,тиннитус,травма,год,частота,шум,эффект,персонаж,внутренний
Мои ключ. слова: звуковой эффект, киноэффект, тиннитус




По TF-IDF я взяла всего 10 слов, и это, кажется, было верным решением - выделились только основные ключевые слова, которые дополняют мои ключевые слова. Иногда они тоже смыслово друг друга повторяют, но точно отражают свои тексты. На вид, думаю, это лучший результат.

In [ ]:
def get_patterns(df):
    pos_patterns = []
    for keywords in df['ideal'].values:
        keyword = keywords.split(',')
        for word in keyword:
            tokens = word.split(' ')
            pos = [m.parse(t)[0].tag.POS for t in tokens]
            if None in pos:
                pattern = ''
            else:
                pattern = '+'.join(pos)
            pos_patterns.append(pattern)
    return set(pos_patterns)
pos_patterns = get_patterns(df)
pos_patterns

{'', 'NOUN+NOUN', 'NOUN+PREP+NOUN', 'VERB'}

Получается, как и на вид, морфологические шаблоны ключевых слов - именные группы из существительных и одиночные глаголы.

In [24]:
def filter(keywords, pos_patterns):
    filtered = []
    for word in keywords:
        tokens = word.split(' ')
        pos = [m.parse(t)[0].tag.POS for t in tokens]
        if None in pos:
            pattern = ''
        else:
            pattern = '+'.join(pos)
        if pattern in pos_patterns:
            filtered.append(word)
    return filtered

In [25]:
rake_filtered = [filter(words, pos_patterns) for words in df['RAKE']]
textrank_filtered = [filter(words, pos_patterns) for words in df['TextRank']]
tfidf_filtered = [filter(words, pos_patterns) for words in df['TF-IDF']]